In [1]:
import carla
import numpy as np
import time

#connected with carla
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)

# load the world
world = client.get_world()
blueprint_library = world.get_blueprint_library()

# choose the car
vehicle_bp = blueprint_library.find('vehicle.tesla.model3')
spawn_point = world.get_map().get_spawn_points()[75]
vehicle = world.spawn_actor(vehicle_bp, spawn_point)

# the initial state
state = np.array([spawn_point.location.x, spawn_point.location.y, spawn_point.location.z, 0, 0, 0])  # X, Y, Z, Vx, Vy, Vz


In [2]:
#adding sensors
gps_bp = blueprint_library.find('sensor.other.gnss')
imu_bp = blueprint_library.find('sensor.other.imu')
lidar_bp = blueprint_library.find('sensor.lidar.ray_cast')

gps = world.spawn_actor(gps_bp, carla.Transform(), attach_to=vehicle)
imu = world.spawn_actor(imu_bp, carla.Transform(), attach_to=vehicle)
lidar = world.spawn_actor(lidar_bp, carla.Transform(carla.Location(z=2.0)), attach_to=vehicle)


In [3]:
#variables to store data
gps_data = None
imu_data = (0,0,0)
lidar_data = []

In [4]:
# GPS FUNC
def gps_callback(data):
    global gps_data
    gps_data = (data.latitude, data.longitude, data.altitude)

# IMU FUNC
def imu_callback(data):
    global imu_data
    imu_data = (data.accelerometer.x, data.accelerometer.y, data.accelerometer.z)

# LiDAR FUNC
def lidar_callback(data):
    global lidar_data
    points = np.frombuffer(data.raw_data, dtype=np.float32)
    points = np.reshape(points, (-1, 4))  
    lidar_data = points

In [5]:
#Calling back the functins
gps.listen(gps_callback)
imu.listen(imu_callback)
lidar.listen(lidar_callback)

In [6]:
 # EKF Model
def predict(state, imu_data, dt):
    ax, ay, az = imu_data
    state[0] += state[3] * dt  #  X new
    state[1] += state[4] * dt  #  Y new
    state[2] += state[5] * dt  #  Z new
    state[3] += ax * dt  #  Vx new
    state[4] += ay * dt  #  Vy new
    state[5] += az * dt  #  Vz new
    return state

def update_with_gps(state, gps_data):
    lat, lon, alt = gps_data
    state[0] = lat  
    state[1] = lon  
    return state
def update_with_lidar(state, lidar_data):
    if len(lidar_data) > 0:
        
        distance = len(lidar_data) * 0.01  
        state[0] += distance * np.cos(state[2])  
        state[1] += distance * np.sin(state[2])  
        print(lidar_data)
    return state


In [7]:
# def detect_obstacle(lidar_data):
#     threshold_distance = 5.0  
#     for point in lidar_data:
#         distance = np.sqrt(point[0]**2 + point[1]**2 + point[2]**2)**0.5
#         if distance < threshold_distance:
#             return True  
#     return False  

In [ ]:
while True:
    dt = 0.1  

    # get the data from the sensors
    gps_data = vehicle.get_transform().location  
    gps_data = (gps_data.x, gps_data.y, gps_data.z)  

    # EKF
    if gps_data is not None and imu_data is not None:
        state = predict(state, imu_data, dt)  
        state = update_with_gps(state, gps_data)  
        state = update_with_lidar(state, lidar_data)  

        

    vehicle.set_autopilot(True)  

    
    print("Current state:", state)
    print("GPS Data:", gps_data)

    time.sleep(dt)


[[ 9.458203    0.43876952 -2.0014236   0.9620293 ]
 [ 9.357988    1.4416796  -2.0014236   0.9620292 ]
 [ 9.232109    2.101914   -2.0014236   0.96202934]
 [ 8.956347    3.0713475  -2.0014236   0.96202946]
 [ 8.841709    3.387324   -2.0014236   0.96202934]
 [ 8.431611    4.307988   -2.0014236   0.96202916]
 [ 7.92584     5.1798825  -2.0014236   0.9620293 ]
 [ 7.7370114   5.4579296  -2.0014236   0.9620293 ]
 [ 7.112988    6.2494335  -2.0014236   0.96202934]
 [ 6.8867188   6.4979296  -2.0014236   0.96202934]
 [ 6.651738    6.738301   -2.0014236   0.9620293 ]
 [ 6.408398    6.970156   -2.0014236   0.9620292 ]
 [ 6.156963    7.193203   -2.0014236   0.9620293 ]
 [ 8.457285    0.9971289  -2.0014236   0.96561354]
 [ 8.365234    1.5945507  -2.0014236   0.9656136 ]
 [ 8.303359    1.8904492  -2.0014236   0.9656136 ]
 [ 8.231035    2.1839647  -2.0014236   0.9656136 ]
 [ 7.952236    3.046582   -2.0014236   0.9656136 ]
 [ 7.8391013   3.3268359  -2.0014236   0.96561366]
 [ 7.2893653   4.402832   -2.00

Current state: [ 1.19324878e+02 -2.00990153e+02  2.75307149e-01 -7.99747556e-03
  2.00973870e-04  9.81000137e-01]
GPS Data: (0.001810629049444401, 0.0010538458497971503, 0.0014237022260203958)
[[-4.7674217   8.178965   -2.001132    0.96203476]
 [-5.6147265   7.622226   -2.001132    0.9620348 ]
 [-5.883291    7.416914   -2.001132    0.96203476]
 ...
 [-0.7878906  -0.5811914  -0.56523836  0.99548817]
 [-0.77065426 -0.6121484  -0.5682054   0.99546456]
 [-2.6354003  -2.251328   -2.001132    0.9841183 ]]
Current state: [ 1.21354513e+02 -1.99975116e+02  3.73407162e-01 -6.99779112e-03
  1.75852136e-04  1.97513866e+00]
GPS Data: (117.31358337402344, -201.5583038330078, 0.000938205688726157)
[[ 3.7350683  -8.699042   -2.0011322   0.9620347 ]
 [ 5.2408104  -7.884023   -2.0011322   0.96203476]
 [ 5.5228906  -7.689062   -2.0011322   0.9620347 ]
 ...
 [ 1.8926171  -0.6528711  -1.1558769   0.99079555]
 [ 1.926455   -0.5873828  -1.1627935   0.9907408 ]
 [ 2.0358105  -0.2348828  -1.1831714   0.9905792

Current state: [ 1.15478142e+02 -2.04242357e+02  1.70289580e+01  3.18217689e+00
  4.19427584e-03  1.86291368e+01]
GPS Data: (116.15550231933594, -201.58740234375, 0.0014851188752800226)
[[-8.881171    3.2023046  -1.9956063   0.9621375 ]
 [-8.990703    2.8792772  -1.9955333   0.96213895]
 [-9.088554    2.5524414  -1.9954681   0.96214026]
 ...
 [-0.95501953  0.08914062 -0.55377454  0.9955796 ]
 [-0.95751953  0.01992187 -0.5529399   0.9955862 ]
 [-0.9572656  -0.04929687 -0.5534099   0.9955825 ]]
Current state: [ 1.18447089e+02 -2.01484241e+02  1.88918716e+01  3.42685369e+00
  2.06385459e-03  1.96088911e+01]
GPS Data: (0.0018109707700375566, 0.0010402442603675943, 0.0015280342195183039)
[[-6.4755955   6.882695   -1.9975603   0.9621011 ]
 [-6.7168064   6.646504   -1.9974123   0.96210384]
 [-7.5929294   5.620586   -1.9968754   0.96211386]
 ...
 [-0.9418066  -0.17576171 -0.55314744  0.99558467]
 [-0.9241699  -0.24185546 -0.5515478   0.9955974 ]
 [-0.91303706 -0.27410156 -0.5504037   0.9956065

[[ 5.284629   -7.8589454  -2.0018787   0.96202093]
 [ 5.5645895  -7.663281   -2.001888    0.9620209 ]
 [ 5.8373632  -7.457695   -2.0018973   0.9620206 ]
 ...
 [ 2.036416   -0.15728515 -1.17922     0.99061054]
 [ 2.0331542  -0.08343749 -1.1748248   0.9906454 ]
 [ 2.0219238  -0.01003906 -1.1673694   0.9907045 ]]
Current state: [ 1.07326946e+02 -1.99329445e+02  5.85342426e+01  4.77533320e+00
  4.03055072e-02  3.42424027e+01]
GPS Data: (108.40695190429688, -201.78219604492188, 0.0017028999282047153)
[[ 9.341855    1.5589257  -2.001988    0.96201885]
 [ 9.279043    1.8972266  -2.0019858   0.9620189 ]
 [ 9.204023    2.2329102  -2.0019834   0.96201897]
 ...
 [ 1.7634082   2.9851952  -2.0017357   0.9841136 ]
 [ 1.6538184   3.0473046  -2.001732    0.9841135 ]
 [ 1.5420605   3.1053514  -2.0017283   0.9841135 ]]
Current state: [ 1.09539378e+02 -2.03835545e+02  6.19584829e+01  4.76716728e+00
  3.25089477e-02  3.52199613e+01]
GPS Data: (107.83100128173828, -201.79666137695312, 0.0016772269736975431

[[ 9.342783   -1.5560741  -2.0020866   0.96201706]
 [ 9.455136   -0.55402344 -2.0020633   0.9620176 ]
 [ 9.468769   -0.21808593 -2.002049    0.96201783]
 ...
 [ 2.9646485   1.7973828  -2.001661    0.98411435]
 [ 2.6804004   2.1989257  -2.0016675   0.9841143 ]
 [ 2.6006737   2.2926757  -2.0016692   0.9841143 ]]
Current state: [ 1.04023066e+02 -2.04550562e+02  9.28802403e+01  4.43063573e+00
  8.95886476e-03  4.30760617e+01]
GPS Data: (103.47804260253906, -201.90614318847656, 0.0017275428399443626)
[[ 8.780829    3.5044725  -1.9984556   0.96208453]
 [ 8.352627    4.429297   -1.998463    0.9620844 ]
 [ 8.188037    4.72668    -1.998469    0.9620843 ]
 [ 7.827734    5.3030076  -1.9985777   0.96208227]
 [ 7.2124023   6.115527   -1.9988369   0.96207744]
 [ 6.755029    6.6185155  -1.9990296   0.962074  ]
 [ 5.738066    7.519902   -1.9994581   0.9620658 ]
 [ 5.1835837   7.913594   -1.9996917   0.96206164]
 [ 4.562422    8.195      -1.9826232   0.9623792 ]
 [ 4.1873536   8.201679   -1.9465498   0

...

[[-3.84542     8.207617   -1.9158943   0.9636219 ]
 [-4.2117968   8.201504   -1.9488711   0.9630076 ]
 [-4.5913477   8.194707   -1.9855523   0.9623247 ]
 ...
 [-0.9543359  -0.09669922 -0.553806    0.99557936]
 [-0.92701167 -0.23283203 -0.5518248   0.9955951 ]
 [-0.91611326 -0.26578125 -0.5507249   0.99560386]]
Current state: [ 1.05298565e+02 -2.02554112e+02  1.75171419e+06 -1.23380637e+01
 -2.32112073e+01  5.87490750e+03]
GPS Data: (101.90262603759766, -201.9457550048828, 0.0016422080807387829)
[[-3.84542     8.207617   -1.9158943   0.9636219 ]
 [-4.2117968   8.201504   -1.9488711   0.9630076 ]
 [-4.5913477   8.194707   -1.9855523   0.9623247 ]
 ...
 [-0.9543359  -0.09669922 -0.553806    0.99557936]
 [-0.92701167 -0.23283203 -0.5518248   0.9955951 ]
 [-0.91611326 -0.26578125 -0.5507249   0.99560386]]
Current state: [ 9.84991084e+01 -2.01381337e+02  1.75230168e+06 -1.23401569e+01
 -2.32151560e+01  5.87589250e+03]
GPS Data: (101.90262603759766, -201.9457550048828, 0.0016422080807387829)


[[-3.84542     8.207617   -1.9158943   0.9636219 ]
 [-4.2117968   8.201504   -1.9488711   0.9630076 ]
 [-4.5913477   8.194707   -1.9855523   0.9623247 ]
 ...
 [-0.9543359  -0.09669922 -0.553806    0.99557936]
 [-0.92701167 -0.23283203 -0.5518248   0.9955951 ]
 [-0.91611326 -0.26578125 -0.5507249   0.99560386]]
Current state: [ 9.96835011e+01 -2.04587342e+02  1.76171493e+06 -1.23736472e+01
 -2.32783342e+01  5.89165253e+03]
GPS Data: (101.90262603759766, -201.9457550048828, 0.0016422080807387829)
[[-3.84542     8.207617   -1.9158943   0.9636219 ]
 [-4.2117968   8.201504   -1.9488711   0.9630076 ]
 [-4.5913477   8.194707   -1.9855523   0.9623247 ]
 ...
 [-0.9543359  -0.09669922 -0.553806    0.99557936]
 [-0.92701167 -0.23283203 -0.5518248   0.9955951 ]
 [-0.91611326 -0.26578125 -0.5507249   0.99560386]]
Current state: [ 9.90207542e+01 -2.00049096e+02  1.76230409e+06 -1.23757403e+01
 -2.32822829e+01  5.89263753e+03]
GPS Data: (101.90262603759766, -201.9457550048828, 0.0016422080807387829)


[[-3.84542     8.207617   -1.9158943   0.9636219 ]
 [-4.2117968   8.201504   -1.9488711   0.9630076 ]
 [-4.5913477   8.194707   -1.9855523   0.9623247 ]
 ...
 [-0.9543359  -0.09669922 -0.553806    0.99557936]
 [-0.92701167 -0.23283203 -0.5518248   0.9955951 ]
 [-0.91611326 -0.26578125 -0.5507249   0.99560386]]
Current state: [ 1.03684850e+02 -2.04899766e+02  1.77174413e+06 -1.24092306e+01
 -2.33454611e+01  5.90839755e+03]
GPS Data: (101.90262603759766, -201.9457550048828, 0.0016422080807387829)
[[-3.84542     8.207617   -1.9158943   0.9636219 ]
 [-4.2117968   8.201504   -1.9488711   0.9630076 ]
 [-4.5913477   8.194707   -1.9855523   0.9623247 ]
 ...
 [-0.9543359  -0.09669922 -0.553806    0.99557936]
 [-0.92701167 -0.23283203 -0.5518248   0.9955951 ]
 [-0.91611326 -0.26578125 -0.5507249   0.99560386]]
Current state: [ 1.04287384e+02 -2.04438831e+02  1.77233497e+06 -1.24113238e+01
 -2.33494098e+01  5.90938255e+03]
GPS Data: (101.90262603759766, -201.9457550048828, 0.0016422080807387829)


[[-3.84542     8.207617   -1.9158943   0.9636219 ]
 [-4.2117968   8.201504   -1.9488711   0.9630076 ]
 [-4.5913477   8.194707   -1.9855523   0.9623247 ]
 ...
 [-0.9543359  -0.09669922 -0.553806    0.99557936]
 [-0.92701167 -0.23283203 -0.5518248   0.9955951 ]
 [-0.91611326 -0.26578125 -0.5507249   0.99560386]]
Current state: [ 1.04203000e+02 -2.04516899e+02  1.78239432e+06 -1.24469072e+01
 -2.34165366e+01  5.92612757e+03]
GPS Data: (101.90262603759766, -201.9457550048828, 0.0016422080807387829)
[[-3.84542     8.207617   -1.9158943   0.9636219 ]
 [-4.2117968   8.201504   -1.9488711   0.9630076 ]
 [-4.5913477   8.194707   -1.9855523   0.9623247 ]
 ...
 [-0.9543359  -0.09669922 -0.553806    0.99557936]
 [-0.92701167 -0.23283203 -0.5518248   0.9955951 ]
 [-0.91611326 -0.26578125 -0.5507249   0.99560386]]
Current state: [ 1.03304298e+02 -1.98793325e+02  1.78298693e+06 -1.24490004e+01
 -2.34204853e+01  5.92711258e+03]
GPS Data: (101.90262603759766, -201.9457550048828, 0.0016422080807387829)


[[-3.84542     8.207617   -1.9158943   0.9636219 ]
 [-4.2117968   8.201504   -1.9488711   0.9630076 ]
 [-4.5913477   8.194707   -1.9855523   0.9623247 ]
 ...
 [-0.9543359  -0.09669922 -0.553806    0.99557936]
 [-0.92701167 -0.23283203 -0.5518248   0.9955951 ]
 [-0.91611326 -0.26578125 -0.5507249   0.99560386]]
Current state: [ 9.92244311e+01 -1.99770951e+02  1.79248213e+06 -1.24824907e+01
 -2.34836635e+01  5.94287260e+03]
GPS Data: (101.90262603759766, -201.9457550048828, 0.0016422080807387829)
[[-3.84542     8.207617   -1.9158943   0.9636219 ]
 [-4.2117968   8.201504   -1.9488711   0.9630076 ]
 [-4.5913477   8.194707   -1.9855523   0.9623247 ]
 ...
 [-0.9543359  -0.09669922 -0.553806    0.99557936]
 [-0.92701167 -0.23283203 -0.5518248   0.9955951 ]
 [-0.91611326 -0.26578125 -0.5507249   0.99560386]]
Current state: [ 1.05310843e+02 -2.02481065e+02  1.79307642e+06 -1.24845838e+01
 -2.34876122e+01  5.94385760e+03]
GPS Data: (101.90262603759766, -201.9457550048828, 0.0016422080807387829)


In [ ]:
gps.destroy()
imu.destroy()
lidar.destroy()
vehicle.destroy()


False